In [ ]:
import os
import pandas as pd

In [ ]:
### still going to need to make new dir for copies ###

data_dir = "./Data/EMG_data_for_gestures-master/"
subj_nums = os.listdir(data_dir)
subj_data = {n : os.listdir(data_dir + n) for n in subj_nums}

In [ ]:
for n in subj_nums:
    print(os.listdir(data_dir + n + "/"))

In [ ]:
# looking at this subject
tst_subj = "03"

In [ ]:
tst_path_0 = data_dir + tst_subj + "/" + subj_data[tst_subj][0]
tt0 = pd.read_table(tst_path_0)

In [ ]:
tst_path_1 = data_dir + tst_subj + "/" + subj_data[tst_subj][1]
tt1 = pd.read_table(tst_path_1)

In [ ]:
tst_path_1 # example path

In [ ]:
# rename "time" variable to "time_ms"
# rename "class" variable to "gesture"
colnames = list(tt0.columns)
colnames[0] = "time_ms"
colnames[-1] = "gesture"
tt0.columns = colnames
tt1.columns = colnames

In [ ]:
def mod_col_names(full_frame):
    """modify column names of full_frame prior to any other manipulation"""
    # rename "time" variable to "time_ms"
    # rename "class" variable to "gesture"
    colnames = list(full_frame.columns)
    colnames[0] = "time_ms"
    colnames[-1] = "gesture"
    full_frame.columns = colnames
    return full_frame

In [ ]:
def find_break_pts(gesture, t = 1000):
    """find and return index for moments in data frame with > t ms between pt and next pt"""
    nrow = gesture.shape[0]
    brk_pts = [i-1 for i in range(2, nrow) if gesture.iloc[i].time_ms - gesture.iloc[i-1].time_ms > t]
    return brk_pts

In [ ]:
def zero_time(seg_df):
    """take previously segmented DataFrame by gesture and set time_ms to start at 0"""
    end_time = min(seg_df.time_ms)
    seg_df.time_ms = seg_df.time_ms - end_time
    return seg_df

In [ ]:
def gest_dict(filepath):
    """return dict where key is gesture # and value is indiv DataFrames for that gesture"""
    df = pd.read_table(filepath)
    df = mod_col_names(df)
    gests = df.gesture.unique()
    all_gst = dict()
    # seperate gestures in file
    for g in gests:
        # split single gesture df into iterations of gesture g
        brk_pts = find_break_pts(df[df.gesture == g])
        gd = dict() # store sub-dfs created by breaks
        trail = 0 
        for n, b in enumerate(brk_pts):
            if g == 0:
                # treat gest == 0 seperately
                # due to many cases btwn non-0 gests
                gd[n] = df[df.gesture == 0].iloc[trail:b]
                gd[n] = zero_time(gd[n])
                train = b + 1
            else:
                # this will handle all other cases (gesture != 0)
                gd[0] = zero_time(df[df.gesture == g].iloc[:b])
                gd[1] = zero_time(df[df.gesture == g].iloc[b+1:])
            all_gst[g] = gd
    
    return all_gst

In [ ]:
tst = gest_dict(tst_path_1)[6]

In [ ]:
tst_path_1

In [ ]:
def build_new_ref(to_dir, sn, orig_file, g, ct):
    """build a new file path based on attributes of data to be saved; naming convention:
    to_dir - top level dir for cleaned data
    sn - subject number
    orig_file - file name with raw data; 2 per subject
    g - gesture number
    ct - counter for files w/ similar names (e.g. gesture 2, 0 & 1)
    """
    if orig_file.startswith("1"):
        new_ref = to_dir + sn + "/1/" + g + "_" + ct
    else:
        new_ref = to_dir + sn + "/2/" + g + "_" + ct
    return new_ref

In [ ]:
for j in subj_nums:
    # get files 1 and 2 for subject j
    fp0 = data_dir + j + "/" + subj_data[j][0]
    fp1 = data_dir + j + "/" + subj_data[j][1]
    gest_dict(fp0)

In [ ]:
# check if residual movement from gesture persists to 0 range

In [ ]:
subj_data["30"][1].startswith("1")